<a href="https://colab.research.google.com/github/isadays/DecisionTreeFixed/blob/main/Test_solution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np, joblib
from sklearn.datasets import make_classification
from sklearn.tree import DecisionTreeClassifier

# 1) Generate artificial classification data
X, y = make_classification(
    n_samples=200, n_features=5, n_informative=3,
    n_redundant=0, n_classes=3, random_state=42
)

# 2) Train & save under the current sklearn
clf = DecisionTreeClassifier(random_state=0)
clf.fit(X, y)
MODEL_PATH = "synthetic_tree.joblib"
joblib.dump(clf, MODEL_PATH)
print("✅ Trained & saved model under sklearn", __import__("sklearn").__version__)


In [ ]:
from IPython import get_ipython
import importlib, sklearn

# (A1) Downgrade to 1.2.2 in this kernel
get_ipython().run_line_magic("pip", "install --quiet scikit-learn==1.2.2")
importlib.reload(sklearn)
print("\n--- A) Running sklearn", sklearn.__version__, "(<1.3)")

# (A2) Subclass-patch Tree to inject the missing field on unpickle
import sklearn.tree._tree as _tm
_orig_Tree = _tm.Tree

class PatchedTree(_orig_Tree):
    def __setstate__(self, state):
        nodes = state.get("nodes")
        if nodes is not None and "missing_go_to_left" not in nodes.dtype.names:
            descr = nodes.dtype.descr + [("missing_go_to_left", "u1")]
            dt    = np.dtype(descr, align=False)
            new   = np.zeros(nodes.shape, dtype=dt)
            for f in nodes.dtype.names:
                new[f] = nodes[f]
            new["missing_go_to_left"] = 0
            state["nodes"] = new
        super(PatchedTree, self).__setstate__(state)

_tm.Tree = PatchedTree
print("Applied Tree subclass for legacy unpickling")

clf_old = joblib.load(MODEL_PATH)
print("Leaf indices under <1.3:", clf_old.apply(X[:5].astype(float)))



In [ ]:
get_ipython().run_line_magic("pip", "install --quiet scikit-learn==1.5.2")
importlib.reload(sklearn)
print("\n--- B) Running sklearn", sklearn.__version__, "(≥1.3)")

clf_new = joblib.load(MODEL_PATH)
print("Leaf indices under ≥1.3:", clf_new.apply(X[:5].astype(float)))